In [1]:
from torch.utils.data import Dataset, DataLoader
import scipy
import os
from PIL import Image
from torch.utils.data import random_split
from torchvision import transforms

In [2]:
# let's resize by just providing one value. It will resize based on the larger dimension and then let's center crop the image.

transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(), # normlize will work not work on image
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std =[0.229, 0.224, 0.225])
    ]
)

In [3]:
# When building a dataset, you need a data class. It needs to answer 3 questions.

class OxfordFlowersDataset(Dataset):

    # Setup: where to find images and labels
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.img_dir = os.path.join(root_dir, 'jpg')

        # load matlab labels
        labels_mat = scipy.io.loadmat(
            os.path.join(root_dir, 'imagelabels.mat')
        )

        # make it zero index
        self.labels = labels_mat['labels'][0] - 1

        # set the trasformation pipeline
        self.transform = transform

    # How many total samples
    def __len__(self):
        return len(self.labels) 

    # How to get image and label number 'idx'
    def __getitem__(self, idx):
        
        # Build the image file name
        img_name = f'image_{idx+1:05d}.jpg'
        img_path = os.path.join(self.img_dir, img_name)

        # load the image
        image = Image.open(img_path)
        label = self.labels[idx]

        # return the right kind of object
        if self.transform:
            image = self.transform(image)

        return image, label

In [4]:
dataset = OxfordFlowersDataset(root_dir = './data')

In [5]:
# split data into 3 sets - 70/15/15 percent
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

In [6]:
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

In [7]:
print(f"Training: {len(train_dataset)} images")
print(f"Validation: {len(val_dataset)} images")
print(f"Test: {len(test_dataset)} images")

Training: 5732 images
Validation: 1228 images
Test: 1229 images


In [8]:
# now, let's define data loaders
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# we don't need to shuffle validation or test dataset
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [9]:
# let's understand the concept of batches and epocs
batch_count = 0
total_images = 0

for images, labels in train_dataloader:
    batch_count += 1
    total_images += len(images)

    # show the last few batches
    if batch_count >= 178:
        print(f"Batch {batch_count}: {len(images)} images")

print(f"\nTotal batches in one epoch: {batch_count}")
print(f"Total images seen: {total_images}")

Batch 178: 32 images
Batch 179: 32 images
Batch 180: 4 images

Total batches in one epoch: 180
Total images seen: 5732
